# Colab-VSGAN

My repo: [styler00dollar/VSGAN-tensorrt-docker](https://github.com/styler00dollar/VSGAN-tensorrt-docker/)

Since it was tried to keep things modular and save on install time, you need to pick on what to install yourself.

In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

In [ ]:
# check python version, for me its currently 3.8.10
!python --version

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
#@title Install TensorRT 8.5.x
%cd /content
!sudo apt-get update
!sudo apt-get install libnvinfer8 libnvonnxparsers8 libnvparsers8 libnvinfer-plugin8 libnvinfer-dev libnvonnxparsers-dev libnvparsers-dev libnvinfer-plugin-dev python3-libnvinfer tensorrt python3-libnvinfer-dev

In [ ]:
#@title initial setup
%cd /content/
!sudo rm -rf /workspace
!mkdir -p /workspace/tensorrt
%cd /workspace/tensorrt
!git clone https://github.com/styler00dollar/VSGAN-tensorrt-docker

!pip install kornia opencv-python pytorch-msssim thop einops timm wget

!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/ffmpeg && \
    chmod +x ffmpeg && sudo rm -rf /usr/bin/ffmpeg && mv ffmpeg /usr/bin/ffmpeg

In [ ]:
#@title download all dependencies
%cd /content
!pip uninstall cupy-cuda11x -y

!mkdir vs_precompiled_colab
%cd vs_precompiled_colab
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/vs_precompiled_colab.7z
!7z x vs_precompiled_colab.7z

!mkdir -p /usr/local/lib/vapoursynth/
!mkdir -p /usr/local/lib/x86_64-linux-gnu/

!cp /content/vs_precompiled_colab/usr_local_bin/vspipe /usr/local/bin/vspipe
!mv /content/vs_precompiled_colab/usr_local_lib/* /usr/local/lib
!sudo ldconfig
!chmod +x /usr/local/bin/vspipe

!pip install /content/vs_precompiled_colab/whl/* scenedetect onnxruntime-gpu vsbasicvsrpp vsswinir https://github.com/pytorch/TensorRT/releases/download/v1.3.0/torch_tensorrt-1.3.0-cp38-cp38-linux_x86_64.whl

# upgrading g++ and installing ffms2
!sudo apt install build-essential manpages-dev software-properties-common ffmsindex libffms2-4 libffms2-dev -y
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt update -y && sudo apt install gcc-11 g++-11 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 11
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-11 11

%cd /content
!rm -rf deb_files
!mkdir deb_files
%cd deb_files
!wget http://mirrors.kernel.org/ubuntu/pool/main/g/glibc/locales_2.36-0ubuntu4_all.deb \
    http://mirrors.kernel.org/ubuntu/pool/main/g/glibc/libc6_2.36-0ubuntu4_amd64.deb \
    http://mirrors.kernel.org/ubuntu/pool/main/g/glibc/libc6-dev_2.36-0ubuntu4_amd64.deb \
    http://mirrors.kernel.org/ubuntu/pool/main/g/glibc/libc-bin_2.36-0ubuntu4_amd64.deb \
    http://mirrors.kernel.org/ubuntu/pool/main/g/glibc/libc-dev-bin_2.36-0ubuntu4_amd64.deb \
    http://mirrors.kernel.org/ubuntu/pool/main/libn/libnsl/libnsl2_1.3.0-2build2_amd64.deb \
    http://mirrors.kernel.org/ubuntu/pool/main/libn/libnsl/libnsl-dev_1.3.0-2build2_amd64.deb \
    http://mirrors.kernel.org/ubuntu/pool/main/libt/libtirpc/libtirpc3_1.3.3+ds-1_amd64.deb \
    http://mirrors.kernel.org/ubuntu/pool/main/libt/libtirpc/libtirpc-common_1.3.3+ds-1_all.deb \
    http://mirrors.kernel.org/ubuntu/pool/main/libt/libtirpc/libtirpc-dev_1.3.3+ds-1_amd64.deb \
    http://mirrors.kernel.org/ubuntu/pool/main/r/rpcsvc-proto/rpcsvc-proto_1.4.2-0ubuntu6_amd64.deb && \
    dpkg --force all -i *.deb

In [ ]:
#@title ffmpeg (nix)
#@markdown Get ffmpeg from nix. Use this to get the nix version of ffmpeg-full instead of my self compiled ffmpeg or if you want to use nvenc. The only difference is, that to use this ffmpeg, you need to use `deroot ffmpeg -i ...`.
%cd /content
!git clone https://github.com/styler00dollar/nix-on-colab  /tmp/nix-on-colab
!/tmp/nix-on-colab/setup
!deroot nix run nixpkgs
!deroot nix-env -iA nixpkgs.ffmpeg_5-full

# (optional) Other cells for modular install / compiling manually
run them in correct order

In [ ]:
#@title compile vs
# installing vapoursynth related stuff
!apt install autoconf libtool yasm python3.9 python3.9-venv python3.9-dev ffmsindex libffms2-4 libffms2-dev -y
!apt --fix-broken install
# zimg
!wget https://github.com/sekrit-twc/zimg/archive/refs/tags/release-3.0.4.zip && 7z x release-3.0.4.zip
%cd zimg-release-3.0.4
!./autogen.sh && ./configure && make -j4 && make install
# vapoursynth
!pip install Cython -U --force-reinstall
!git clone https://github.com/vapoursynth/vapoursynth
%cd vapoursynth
!./autogen.sh && ./configure && make && make install && cd .. && ldconfig
!ln -s /usr/local/lib/python3.8/site-packages/vapoursynth.so /usr/lib/python3.8/lib-dynload/vapoursynth.so
!pip install vapoursynth

Optional stuff, which you may or may not need:

In [ ]:
#@title install cupy / pycuda
!curl https://colab.chainer.org/install | sh -
!pip install pycuda

In [ ]:
#@title install Torch-TensorRT
# pytorch tensorrt
!pip install https://github.com/pytorch/TensorRT/releases/download/v1.2.0/torch_tensorrt-1.2.0-cp37-cp37m-linux_x86_64.whl

In [ ]:
#@title install onnx
!pip install onnx onnxruntime onnxruntime-gpu

In [ ]:
#@title install Onnx-TensorRT
!pip install nvidia-pyindex nvidia-tensorrt pycuda
%cd /workspace/tensorrt
!git clone https://github.com/onnx/onnx-tensorrt
%cd onnx-tensorrt
!python3 setup.py install

In [ ]:
#@title install swinir
# vs plugings from others
# https://github.com/HolyWu/vs-swinir
!pip install --upgrade vsswinir && python -m vsswinir

In [ ]:
#@title compile mmcv + install vsbasicvsrpp (basicvsrpp/RealBasicVSR)
# https://github.com/HolyWu/vs-basicvsrpp
!pip install --upgrade vsbasicvsrpp && python -m vsbasicvsrpp
# dependencies for RealBasicVSR_x4
# mmedit
!git clone https://github.com/open-mmlab/mmediting.git && cd mmediting && pip install -v -e .
# RealBasicVSR_x4 will download this
!wget "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" -P /root/.cache/torch/hub/checkpoints/

In [ ]:
#@title download pytorch models 
%cd /workspace/tensorrt/
!sh download_models.sh

In [ ]:
#@title install g++11
!sudo apt install build-essential manpages-dev software-properties-common -y
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt update -y && sudo apt install gcc-11 g++-11 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 11
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-11 11

In [ ]:
#@title compile vs-mlrt (needs g++11)
# updating cmake
%cd /content
!wget https://github.com/Kitware/CMake/releases/download/v3.24.0-rc1/cmake-3.24.0-rc1-linux-x86_64.sh
!chmod +x cmake-3.24.0-rc1-linux-x86_64.sh
!sh cmake-3.24.0-rc1-linux-x86_64.sh --skip-license
!cp /content/bin/cmake /usr/bin/cmake
!cp /content/bin/cmake /usr/lib/x86_64-linux-gnu/cmake
!cp /content/bin/cmake /usr/local/bin/cmake
!cp -r /content/share/cmake-3.24 /usr/local/share/
# getting vapoursynth sourcecode
%cd /content
!wget https://github.com/vapoursynth/vapoursynth/archive/refs/tags/R60.zip
!7z x R60.zip
# compiling
%cd /content/
!sudo rm -rf vs-mlrt
!git clone https://github.com/AmusementClub/vs-mlrt
%cd vs-mlrt/vstrt
!mkdir build
%cd build
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 cmake .. -DVAPOURSYNTH_INCLUDE_DIRECTORY=/content/vapoursynth-R60/include
!make
!sudo make install
%cd /content

In [ ]:
#@title compile VMAF (needs compiled vs and g++11)
%cd /content
!apt install nasm -y
!pip install ninja meson

!rm -rf vmaf-2.3.1
!wget https://github.com/Netflix/vmaf/archive/refs/tags/v2.3.1.tar.gz
# VMAF
!tar -xzf  v2.3.1.tar.gz
%cd vmaf-2.3.1/libvmaf/
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 meson build --buildtype release
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 ninja -C build && ninja -C build install

%cd /content
!rm -rf VapourSynth-VMAF
!git clone https://github.com/HomeOfVapourSynthEvolution/VapourSynth-VMAF
%cd VapourSynth-VMAF
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 meson build
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 ninja -C build && ninja -C build install

In [ ]:
#@title compile stuff for ddfi (VFRToCFR, mvtools, fmtconv) (needs compiled vs)
%cd /content
!apt install nasm fftw3-dev -y
!pip install ninja meson

# Vapoursynth-VFRToCFR
!sudo rm -rf Vapoursynth-VFRToCFR
!git clone https://github.com/Irrational-Encoding-Wizardry/Vapoursynth-VFRToCFR && cd Vapoursynth-VFRToCFR && \
    mkdir build && cd build && meson --buildtype release .. && ninja && ninja install

# vapoursynth-mvtools
%cd /content
!sudo rm -rf vapoursynth-mvtools
!git clone https://github.com/dubhater/vapoursynth-mvtools && cd vapoursynth-mvtools && ./autogen.sh && ./configure && make && make install

# fmtconv
%cd /content
!sudo rm -rf fmtconv
!git clone https://github.com/EleonoreMizo/fmtconv && cd fmtconv/build/unix/ && ./autogen.sh && ./configure && make && make install

Vulkan related:

In [ ]:
#@title install vulkan
!sudo apt-get install -y libvulkan-dev nasm
!pip install meson ninja
# upgrading g++
!sudo apt install build-essential manpages-dev software-properties-common -y
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt update -y && sudo apt install gcc-11 g++-11 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 11
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-11 11

In [ ]:
#@title compile rife ncnn (needs compiled vs, VMAF and g++11)
# MISC
%cd /content
!rm -rf vs-miscfilters-obsolete
!git clone https://github.com/vapoursynth/vs-miscfilters-obsolete
%cd vs-miscfilters-obsolete
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 meson build
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 ninja -C build && ninja -C build install
    
# RIFE
%cd /content
!rm -rf VapourSynth-RIFE-ncnn-Vulkan
!git clone https://github.com/styler00dollar/VapourSynth-RIFE-ncnn-Vulkan
%cd VapourSynth-RIFE-ncnn-Vulkan
!git submodule update --init --recursive --depth 1
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 meson build
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 ninja -C build && ninja -C build install

Rendering related:

In [ ]:
#@title install x264
# render tools
!sudo apt install x264 -y

In [ ]:
#@title compile x265
!sudo apt-get install -y numactl
# updating cmake
%cd /content
!wget https://github.com/Kitware/CMake/releases/download/v3.23.0-rc1/cmake-3.23.0-rc1-linux-x86_64.sh
!chmod +x cmake-3.23.0-rc1-linux-x86_64.sh
!sh cmake-3.23.0-rc1-linux-x86_64.sh --skip-license
!cp /content/bin/cmake /usr/bin/cmake
!cp /content/bin/cmake /usr/lib/x86_64-linux-gnu/cmake
!cp /content/bin/cmake /usr/local/bin/cmake
!cp -r /content/share/cmake-3.23 /usr/local/share/
# upgrading g++
!sudo apt install build-essential manpages-dev software-properties-common -y
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt update -y && sudo apt install gcc-11 g++-11 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 11
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-11 11
# compile x265
%cd /content
!sudo rm -rf x265
!git clone https://github.com/AmusementClub/x265
%cd x265/source/
!mkdir build
%cd build
!cmake .. -DNATIVE_BUILD=ON -DSTATIC_LINK_CRT=ON -DENABLE_AVISYNTH=OFF
!make 
!sudo make install
!cp /content/x265/source/build/x265 /usr/bin/x265 
!cp /content/x265/source/build/x265 /usr/local/bin/x265

`Runtime -> Restart runtime`

# Render

In [ ]:
# if you want to use vs-mlrt, convert model into engine, you can find models in /content/models
# after creating the model, you can set the path in inference.py
%cd /content
!/usr/src/tensorrt/bin/trtexec --fp16 --onnx=/content/model.onnx --minShapes=input:1x3x8x8 --optShapes=input:1x3x720x1280 --maxShapes=input:1x3x1080x1920 --saveEngine=/content/model.engine --tacticSources=+CUDNN,+CUBLAS,+CUBLAS_LT

# Normal inference

sys.path.append('/workspace/tensorrt/VSGAN-tensorrt-docker/')

The default installation by downloading dependencies does not include vsbasicvsrpp and 

In [ ]:
#@title inference.py
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/inference.py
import sys

sys.path.append("/workspace/tensorrt/VSGAN-tensorrt-docker/")
from inference_config import inference_clip

video_path = "/workspace/tensorrt/VSGAN-tensorrt-docker/720.mkv"
clip = inference_clip(video_path)
clip.set_output()

In [ ]:
#@title inference_config.py
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/inference_config.py
import sys
import os
sys.path.append("/workspace/tensorrt/VSGAN-tensorrt-docker/")
import vapoursynth as vs

# video imports
from src.vfi_inference import vfi_inference
from src.vfi_model import video_model

from src.rife import RIFE
from src.IFRNet import IFRNet
from src.GMFupSS import GMFupSS
from src.GMFSS_union import GMFSS_union
from src.eisai import EISAI
from src.film import FILM
from src.M2M import M2M
from src.sepconv_enhanced import sepconv
from src.IFUNet import IFUNet
from src.stmfnet import STMFNet
from src.rife_trt import rife_trt

# upscale imports
from src.upscale_inference import upscale_inference
from src.pan import PAN_inference
from src.realbasicvsr import realbasicvsr_inference
from src.egvsr import egvsr_inference
from src.cugan import cugan_inference
from vsbasicvsrpp import BasicVSRPP
from vsswinir import SwinIR
from src.SRVGGNetCompact import compact_inference

# from src.esrgan import ESRGAN_inference

# image processing imports
from src.scunet import scunet_inference

from src.scene_detect import scene_detect

core = vs.core
vs_api_below4 = vs.__api_version__.api_major < 4
core = vs.core
core.num_threads = 4  # can influence ram usage
# only needed if you are inside docker
core.std.LoadPlugin(path="/usr/lib/x86_64-linux-gnu/libffms2.so")
core.std.LoadPlugin(path="/usr/local/lib/libvstrt.so")
core.std.LoadPlugin(path="/usr/local/lib/libscxvid.so")
core.std.LoadPlugin(path="/usr/local/lib/libwwxd.so")


def inference_clip(video_path="", clip=None):
    # ddfi is passing clip
    if clip is None:
        # cfr video
        clip = core.ffms2.Source(source=video_path, cache=False)
        # vfr video
        # clip = core.ffms2.Source(source=video_path, fpsnum = 24000, fpsden = 1001, cache=False)
        # vfr video (automatically set num and den)
        # clip = core.ffms2.Source(source=video_path, fpsnum = -1, fpsden = 1, cache=False)

        # lsmash
        # clip = core.lsmas.LWLibavSource(source=video_path)
        
        # resizing with descale
        # Debilinear, Debicubic, Delanczos, Despline16, Despline36, Despline64, Descale
        #clip = core.descale.Debilinear(clip, 1280, 720)

    ###############################################
    # SIMILARITY
    # Set properties in clip for it to be applied
    # SSIM for deduplication in frame interpolation

    # offs1 = core.std.BlankClip(clip, length=1) + clip[:-1]
    # offs1 = core.std.CopyFrameProps(offs1, clip)
    # 0 = PSNR, 1 = PSNR-HVS, 2 = SSIM, 3 = MS-SSIM, 4 = CIEDE2000
    # clip = core.vmaf.Metric(clip, offs1, 2)

    # SCENE DETECT
    # clip = core.misc.SCDetect(clip=clip, threshold=0.100)
    # clip = core.scxvid.Scxvid(clip, use_slices=True) # todo
    # clip = core.wwxd.WWXD(clip=clip) # todo

    # model based scene detect needs RGBS as input
    # clip = scene_detect(clip, model_name="efficientnetv2_b0", thresh=0.98, fp16=False)

    # dedup (requires you to call "vspipe parse.py -p ." to generate infos_running.txt and tmp.txt)
    #from src.dedup import get_dedup_frames
    #frames_duplicated, frames_duplicating = get_dedup_frames()
    #clip = core.std.DeleteFrames(clip, frames_duplicated)
    # do upscaling here
    #clip = core.std.DuplicateFrames(clip, frames_duplicating)
    ###############################################
    # COLORSPACE
    ###############################################

    # convert colorspace
    clip = vs.core.resize.Bicubic(clip, format=vs.RGBS, matrix_in_s="709")
    # convert colorspace + resizing
    # clip = vs.core.resize.Bicubic(
    #    clip, width=1280, height=720, format=vs.RGBS, matrix_in_s="709"
    # )

    ###############################################
    # MODELS
    ###############################################
    # in rare cases it can happen that image range is not 0-1 and that resulting in big visual problems, clamp input
    clip = core.akarin.Expr(clip, "x 0 1 clamp")
    # clip = core.std.Limiter(clip, max=1, planes=[0,1,2])
    #clip = scene_detect(clip, model_name="efficientnetv2_b0", thresh=0.98)

    ######
    # VFI
    ######

    # VFI example for jit models
    # clip = video_model(clip, fp16=False, model_path="/workspace/rvpV1_105661_G.pt")

    # Rife: model "rife40" up to "rife46" and "sudo_rife4"
    # model_inference = RIFE(
    #    scale=1, fastmode=True, ensemble=False, model_version="rife46", fp16=True
    # )

    # IFRNet: model="small" or "large"
    # model_inference = IFRNet(model="small", fp16=False)

    # model_inference = GMFupSS(partial_fp16=False)

    model_inference = GMFSS_union(partial_fp16=True)

    # model_inference = EISAI() # 960x540

    # FILM: model_choise="style", "l1" or "vgg"
    # model_inference = FILM(model_choise="vgg")

    # model_inference = M2M()

    # model_inference = sepconv() # only 2x supported because architecture only outputs one image

    # model_inference = IFUNet()

    # model_inference = STMFNet()  # only 2x supported because architecture only outputs one image

    clip = vfi_inference(
        model_inference=model_inference, clip=clip, multi=2, metric_thresh=0.999
    )

    # clip = rife_trt(clip, multi = 2, scale = 1.0, device_id = 0, num_streams = 2, engine_path = "/workspace/tensorrt/rife46.engine")

    ######
    # UPSCALING WITH TENSORRT
    ######
    # vs-mlrt (you need to create the engine yourself, read the readme)
    """
    clip = core.trt.Model(
        clip,
        engine_path="/workspace/tensorrt/cugan.engine",
        #tilesize=[854, 480],
        overlap=[0 ,0],
        num_streams=4,
    )
    """

    # vs-mlrt (DPIR)
    # DPIR does need an extra channel
    # strength = 10.0
    # noise_level = clip.std.BlankClip(format=vs.GRAYS, color=strength / 100)
    # clip = core.trt.Model(
    #    [clip, noise_level],
    #    engine_path="dpir.engine",
    #    tilesize=[1280, 720],
    #    num_streams=2,
    # )

    ######
    # CUDA (upscaling/denoising)
    # if possible, use mlrt from above instead due to speed
    ######

    # upscale_model_inference = PAN_inference(scale = 2, fp16 = True)

    # upscale_model_inference = egvsr_inference(scale=4)

    # CUGAN: kind_model="no_denoise", "conservative" or "denoise3x"
    # upscale_model_inference = cugan_inference(fp16=True,scale=2,kind_model="no_denoise")

    # upscale_model_inference = scunet_inference(fp16 = True)

    # WARNING: FOR NOW NOT AVAILABLE DUE TO COMPATIBILITY ISSUES, use mlrt instead
    # ESRGAN: tta is in the range between 1 and 7
    # upscale_model_inference = ESRGAN_inference(model_path="/workspace/tensorrt/models/RealESRGAN_x4plus_anime_6B.pth", fp16=False, tta=False, tta_mode=1)

    # Compact: no tiling allowed due to onnx-tensorrt not allowing dynamic shapes, use mlrt instead though
    # upscale_model_inference = compact_inference(scale=2, fp16=True, clip=clip)

    # upscale_model_inference = realbasicvsr_inference(fp16=True)

    # clip = upscale_inference(upscale_model_inference, clip, tile_x=512, tile_y=512, tile_pad=10, pre_pad=0)

    ######
    # external vs plugins
    ######

    # BasicVSR++
    # 0 = REDS, 1 = Vimeo-90K (BI), 2 = Vimeo-90K (BD), 3 = NTIRE 2021 - Track 1, 4 = NTIRE 2021 - Track 2, 5 = NTIRE 2021 - Track 3
    # clip = BasicVSRPP(
    #    clip,
    #    model=1,
    #    interval=30,
    #    tile_x=0,
    #    tile_y=0,
    #    tile_pad=16,
    #    device_type="cuda",
    #    device_index=0,
    #    fp16=False,
    #    cpu_cache=False,
    # )

    # SwinIR
    # clip = SwinIR(clip, task="lightweight_sr", scale=2)

    ###############################################
    # ncnn (works in docker, but only on linux, because wsl on windows does not support vulkan)
    ###############################################

    # Rife ncnn (C++)
    # Model list can be found in https://github.com/styler00dollar/VapourSynth-RIFE-ncnn-Vulkan
    # clip = core.misc.SCDetect(clip=clip, threshold=0.100)
    # clip = core.rife.RIFE(
    #    clip,
    #    model=9,
    #    factor_num=2,
    #    gpu_id=0,
    #    gpu_thread=4,
    #    tta=False,
    #    uhd=False,
    #    skip=True,
    #    sc=True,
    # )

    ######
    # DDFI
    # you need to use 8x interp for this
    ######
    # advanced example with pytorch vfi + dedup + scene change + upscaling

    # offs1 = core.std.BlankClip(clip, length=1) + clip[:-1]
    # offs1 = core.std.CopyFrameProps(offs1, clip)
    # clip = core.vmaf.Metric(clip, offs1, 2)
    # clip = core.resize.Bicubic(clip, width=1280, height=720, format=vs.RGBS, matrix_in=1)

    # clip = core.misc.SCDetect(clip=clip, threshold=0.100)

    # model_inference = GMFupSS(partial_fp16=True)
    # clip = vfi_inference(
    #     model_inference=model_inference, clip=clip, multi=8, metric_thresh=0.999
    # )

    # clip = vs.core.resize.Bicubic(clip, format=vs.YUV420P8, matrix_s="709")
    # offs1 = core.std.BlankClip(clip, length=1) + clip[:-1]
    # offs1 = core.std.CopyFrameProps(offs1, clip)
    # clip = core.vmaf.Metric(clip, offs1, 2)
    # clip = vs.core.resize.Bicubic(clip, format=vs.RGBS, matrix_in_s="709")

    # clip = core.trt.Model(
    #     clip,
    #     engine_path="/content/model.engine",
    #     num_streams=3,
    # )

    ###############################################
    # OUTPUT
    ###############################################
    clip = vs.core.resize.Bicubic(clip, format=vs.YUV420P8, matrix_s="709")
    return clip

In [ ]:
# self compiled ffmpeg / default
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!vspipe -c y4m inference.py - | ffmpeg -i pipe: /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv -y

In [ ]:
# with nix ffmpeg
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!vspipe -c y4m inference.py - | deroot ffmpeg -i pipe: -vcodec h264_nvenc /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv -y

In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!vspipe -c y4m inference.py - | x264 - --demuxer y4m -o /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv -y

In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!vspipe -c y4m inference.py - | x265 - --y4m --crf 23 -o /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv -y

# Batch inference
Also uses `inference_config.py` for configuration, but you also need to use `main.py` to iterate over files.

In [ ]:
#@title main.py
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/main.py
import glob
import os
import shutil

input_dir = "/workspace/tensorrt/input/"
tmp_dir = "tmp/"
output_dir = "/workspace/tensorrt/output/"
files = glob.glob(input_dir + "/**/*.mkv", recursive=True)
files.sort()

for f in files:
    # creating folders if they dont exist
    if not os.path.exists(tmp_dir):
        os.mkdir(tmp_dir)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    if os.path.exists(tmp_dir):
        shutil.rmtree(tmp_dir)
        os.mkdir(tmp_dir)

    # paths
    txt_path = os.path.join(tmp_dir, "tmp.txt")
    out_render_path = os.path.join(
        output_dir, os.path.splitext(os.path.basename(f))[0] + "_rendered.mkv"
    )
    mux_path = os.path.join(
        output_dir, os.path.splitext(os.path.basename(f))[0] + "_mux.mkv"
    )

    # only needed for dedup
    #os.system(f"vspipe /workspace/tensorrt/parse.py --arg source={f} -p .")

    #os.system(f"vspipe -c y4m inference_batch.py - | ffmpeg -i pipe: {out_render_path}")
    #os.system(
    #    f"ffmpeg -y -loglevel error -i {f} -i {out_render_path}  -map 1 -map 0 -map -0:v -codec copy -max_interleave_delta 0 {mux_path}"
    #)

    os.system(
       f"vspipe -c y4m inference_batch.py --arg source={f} - | ffmpeg -y -i {f} -thread_queue_size 100 -i pipe: -map 1 -map 0 -map -0:v -max_interleave_delta 0 -scodec copy -crf 10 -preset slow {mux_path}"
    )

    # deleting temp files
    #os.remove(txt_path)

In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker
!python main.py

# ddfi

Auto dedup-duplication inference example (More information on what exactly is meant is in [Mr-Z-2697/ddfi-rife](https://github.com/Mr-Z-2697/ddfi-rife).) Also uses `inference_config.py` for configuration, but you also need to use `deduped_vfi.py` to iterate over files.

In [ ]:
#@title parse.py
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/parse.py
import vapoursynth as vs
import os
import functools

core = vs.core
core.num_threads = 4
core.max_cache_size = 4096

core.std.LoadPlugin(path="/usr/lib/x86_64-linux-gnu/libffms2.so")
core.std.LoadPlugin(path="/usr/local/lib/libfmtconv.so")
core.std.LoadPlugin(path="/usr/local/lib/libmvtools.so")

# https://github.com/xyx98/my-vapoursynth-script/blob/master/xvs.py
def props2csv(
    clip: vs.VideoNode,
    props: list,
    titles: list,
    output="info.csv",
    sep="\t",
    charset="utf-8",
    tostring=None,
):
    file = open(output, "w", encoding=charset)
    file.write(sep.join(["n"] + titles))
    tostring = (
        tostring
        if callable(tostring)
        else lambda x: x.decode("utf-8")
        if isinstance(x, bytes)
        else str(x)
    )

    def tocsv(n, f, clip):
        file.write(
            "\n"
            + sep.join(
                [str(n)]
                + [tostring(eval("f.props." + i, globals(), {"f": f})) for i in props]
            )
        )

        return clip
        file.close()

    return core.std.FrameEval(clip, functools.partial(tocsv, clip=clip), prop_src=clip)


clip = core.ffms2.Source(globals()["source"], cache=False)
offs1 = core.std.BlankClip(clip, length=1) + clip[:-1]
offs1 = core.std.CopyFrameProps(offs1, clip)
offs1 = core.vmaf.Metric(clip, offs1, 2)
offs1 = core.std.MakeDiff(offs1, clip)
offs1 = core.fmtc.bitdepth(offs1, bits=16)
offs1 = core.std.Expr(offs1, "x 32768 - abs")
offs1 = core.std.PlaneStats(offs1)
offs1 = props2csv(
    offs1,
    props=["_AbsoluteTime", "float_ssim", "PlaneStatsMax"],
    output=os.path.join(tmp_dir, "infos_running.txt"),
    titles=[],
)
offs1.set_output()

In [ ]:
#@title ddfi.py
#@markdown edit this file to customize interpolation
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/ddfi.py
import sys
sys.path.append("/workspace/tensorrt/VSGAN-tensorrt-docker/")
import vapoursynth as vs
import os
from src.rife import RIFE
from src.IFRNet import IFRNet
from src.GMFupSS import GMFupSS
from src.eisai import EISAI
from src.film import FILM
from src.vfi_inference import vfi_inference

core = vs.core

core.std.LoadPlugin(path="/usr/lib/x86_64-linux-gnu/libffms2.so")
core.std.LoadPlugin(path="/usr/local/lib/libmvtools.so")
core.std.LoadPlugin(path="/usr/local/lib/libvstrt.so")


ssimt = 0.999
pxdifft = 10240
consecutivet = 2
core.num_threads = 4
core.max_cache_size = 4096

tmp_dir = "tmp/"

# frames to delete
def processInfo():
    with open(os.path.join(tmp_dir, "infos_running.txt"), "r") as f:
        lines = [i.split("\t") for i in f][1:]
    for i in range(len(lines)):
        lines[i][0] = int(lines[i][0])
        lines[i][1] = int(float(lines[i][1]) * 1000)
        lines[i][2] = float(lines[i][2])
        lines[i][3] = int(lines[i][3])
    lines.sort()
    startpts = lines[0][1]
    consecutive = 0

    dels = []
    tsv2o = []
    for i in range(len(lines)):
        l = lines[i]
        if l[2] >= ssimt and l[3] <= pxdifft and consecutive < consecutivet:
            consecutive += 1
            dels.append(l[0])
        else:
            consecutive = 0
            tsv2o.append(l[1] - startpts)
    return dels, tsv2o


def newTSgen(tsv2o):
    ts_new = list()
    outfile = open(os.path.join(tmp_dir, "tsv2nX8.txt"), "w", encoding="utf-8")
    ts_o = [i for i in tsv2o][1:]

    for x in range(len(ts_o) - 1):
        ts_new.append(str(float(ts_o[x])))
        for i in range(1, 8):
            ts_new.append(
                str(float(ts_o[x]) + (float(ts_o[x + 1]) - float(ts_o[x])) / 8 * i)
            )
    print("#timestamp format v2", file=outfile)
    for x in range(len(ts_new)):
        print(ts_new[x], file=outfile)
    print(ts_o[len(ts_o) - 1], file=outfile)
    outfile.close()


dels, tsv2o = processInfo()
newTSgen(tsv2o)

with open(os.path.join(tmp_dir, "tmp.txt")) as f:
    video_path = f.readlines()[0]
clip = core.ffms2.Source(video_path)
clip = core.std.DeleteFrames(clip, dels)
sup = core.mv.Super(clip, pel=1, levels=1)
bw = core.mv.Analyse(sup, isb=True, levels=1, truemotion=False)
clip = core.mv.SCDetection(clip, bw, thscd1=200, thscd2=85)


# easy example with ncnn rife
clip = core.resize.Bicubic(clip, format=vs.RGBS, matrix_in=1)
clip = core.misc.SCDetect(clip=clip, threshold=0.100)
clip = core.rife.RIFE(clip, model=9, sc=True, skip=True, multiplier=8)

# example for custom vfi
"""
clip = core.resize.Bicubic(clip, format=vs.RGBS, matrix_in=1)

model_inference = RIFE(
    scale=1, fastmode=False, ensemble=True, model_version="rife46", fp16=False
)
# model_inference = IFRNet(model="small", fp16=False)
# model_inference = GMFupSS()
# model_inference = EISAI() # 960x540
# model_inference = FILM(model_choise="vgg")
clip = vfi_inference(
    model_inference=model_inference, clip=clip, multi=8
)
"""

# advanced example with pytorch vfi + dedup + scene change + upscaling
"""
offs1 = core.std.BlankClip(clip, length=1) + clip[:-1]
offs1 = core.std.CopyFrameProps(offs1, clip)
clip = core.vmaf.Metric(clip, offs1, 2)
clip = core.resize.Bicubic(clip, width=1280, height=720, format=vs.RGBS, matrix_in=1)

clip = core.misc.SCDetect(clip=clip, threshold=0.100)

model_inference = GMFupSS(partial_fp16=True)
clip = vfi_inference(
    model_inference=model_inference, clip=clip, multi=8
)

clip = vs.core.resize.Bicubic(clip, format=vs.YUV420P8, matrix_s="709")
offs1 = core.std.BlankClip(clip, length=1) + clip[:-1]
offs1 = core.std.CopyFrameProps(offs1, clip)
clip = core.vmaf.Metric(clip, offs1, 2)
clip = vs.core.resize.Bicubic(clip, format=vs.RGBS, matrix_in_s="709")

clip = core.trt.Model(
    clip,
    engine_path="/content/model.engine",
    num_streams=3,
)
"""
#######################

clip = core.resize.Bicubic(
    clip, format=vs.YUV420P10, matrix=1, dither_type="error_diffusion"
)
clip = core.vfrtocfr.VFRToCFR(
    clip, os.path.join(tmp_dir, "tsv2nX8.txt"), 192000, 1001, True
)  # 24fps * 8
sup = core.mv.Super(clip)
fw = core.mv.Analyse(sup)
bw = core.mv.Analyse(sup, isb=True)
clip = core.mv.FlowFPS(clip, sup, bw, fw, 60, 1)
clip.set_output()

In [ ]:
#@title deduped_vfi.py
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/deduped_vfi.py
import glob
import os

input_dir = "/workspace/tensorrt/VSGAN-tensorrt-docker/input/"
tmp_dir = "tmp/"
output_dir = "/workspace/tensorrt/VSGAN-tensorrt-docker/output/"
files = glob.glob(input_dir + "/**/*.mkv", recursive=True)
files.sort()

for f in files:
    # creating folders if they dont exist
    if os.path.exists(tmp_dir) == False:
        os.mkdir(tmp_dir)
    if os.path.exists(output_dir) == False:
        os.mkdir(output_dir)

    # paths
    txt_path = os.path.join(tmp_dir, "tmp.txt")
    out_render_path = os.path.join(
        output_dir, os.path.splitext(os.path.basename(f))[0] + "_rendered.mkv"
    )
    mux_path = os.path.join(
        output_dir, os.path.splitext(os.path.basename(f))[0] + "_mux.mkv"
    )

    # writing filepath into temp txt
    # workaround to pass filename parameter
    f_txt = open(txt_path, "w")
    f_txt.write(str(f))
    f_txt.close()

    os.system("vspipe parse.py -p .")
    os.system(
        f"vspipe -c y4m ddfi.py - | ffmpeg -i pipe: -vcodec libsvtav1 -crf 20 {out_render_path}"
    )

    os.system(
        f"ffmpeg -y -loglevel error -i {f} -i {out_render_path}  -map 1 -map 0 -map -0:v -codec copy -max_interleave_delta 0 {mux_path}"
    )

    # deleting temp files
    os.remove(txt_path)

In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!python deduped_vfi.py